In [1]:
from github import Github
import os
import datetime
import pandas as pd
import numpy as np

from dash import Dash, html, dcc, callback, Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"

import dotenv
# Reload the variables in your '.env' file (override the existing variables)
dotenv.load_dotenv(".env", override=True)

# Authentication is defined via github.Auth
from github import Auth

# using an access token
auth = Auth.Token(os.getenv('GITHUB_ACCESS_TOKEN'))
g = Github(auth=auth)

def get_top_repo(topics):
    page=0
    top = 10

    rows_list = []

    if len(topics) > 1:
        return
    # for t in topics:

    t = topics[0]
    repositories = g.search_repositories(query=f'stars:>0 topic:{t} is:public', sort='stars', order='desc')
    print(t, repositories.totalCount) # Not clear why it prints 1000 for totalCount and passes a differnt value in return
    # pages of 30 items. We get the first page
    page0 = repositories.get_page(page)

    row = ()
    for repo in page0:
        row = (t, repo.name, repo.stargazers_count, repo.forks_count, repo.topics, repo.description, repo.language, repo.created_at.date())
        rows_list.append(row)
        df = pd.DataFrame(columns=['topic','repo', 'stars', 'forks', 'topics', 'description', 'language', 'created_at'], data = rows_list)

    return df.head(top), repositories, repositories.totalCount

In [ ]:
search_val = 'microsoft/semantic-kernel'
search_repo = search_val.split('/')[1]
topics = ['llm']